## Ce notebook sert à tout simplement à ajouter au tableau des articles du BSPE une colonne qui distingue la catégorie de l'article 


La particularité de ce notebook est que l'on s'occupe ici de merger 2 dataframes : 

1. Celui que j'ai constitué il y a déjà un bon moment à partir des documents sur <a href = "https://pathexo.societe-mtsi.fr/1844-sommaire-bulletin.html"> le site de la Société Francophone de Médecine Tropicale et Santé Internationale </a> qui s'arrête en 1921

2. Le df fourni par les responsables de Medica où l'on trouve beaucoup plus d'informations. 


* Communication
* Mémoires
* Discussion
* Conférences
* Vie-SPE (Nécrologie, Discours d'ouverture, Correspondances)
* Société de Médecine de l'OUest Africain
* Société des sciences médecinales de Madagascar

In [1]:
import requests
import csv
import pandas as pd
import re

import Structuration_Medica as Medica

IndentationError: expected an indented block (Structuration_Medica.py, line 23)

In [ ]:
Medica

## DF MEDICA

In [ ]:
with open ("MEDICA-BSPE.tsv") as BSPE_Medica:
    print(type(BSPE_Medica))
    df_Medica = pd.read_csv(BSPE_Medica,sep='\t')

In [ ]:
df_Medica

In [ ]:
#je retire la colonne"Unnamed": 
df_Medica.drop('Unnamed: 0',
  axis='columns', inplace=True)

In [ ]:
#Ajout du article-id qui permettra de merger avec df-perso

#Nouvelle colonne pour l'article-id : 
df_Medica["article_id"] = df_Medica.apply(lambda _: '', axis=1)

#Nouvelle colonne pour l'année : 
df_Medica["revue_annee"] = df_Medica.apply(lambda _: '', axis=1)

list(df_Medica.columns)

#Réorganisation des colonnes pour mettre en avant article_id, page,chapitre, année qui sont les éléments que je retrouve dans mon df_preso 
df_Medica = df_Medica[['cote',
 'article_id',
 'page',
 'chapitre',
 'revue_annee',
 'pagecle',
 'refimg',
 'pagimgcle',
 'pagimg',
 'pagimgtxt',
 'pagimgtxtcom']]




In [ ]:
for i in range (len(df_Medica)):
    #Extraction de l'année : 
    df_Medica.iloc[i,4] = re.search(r'\d{4}',df_Medica.iloc[i,0]).group(0)
    
    #Constitution de l'article_id
    df_Medica.iloc[i,1] = "BSPE-"+df_Medica.iloc[i,4]+"-"+df_Medica.iloc[i,2]

In [ ]:
df_Medica

In [ ]:
df_Medica_court = df_Medica[["article_id","chapitre"]]

In [ ]:
df_Medica_court

In [ ]:
if type(df_Medica) is pd.core.frame.DataFrame:
    print ("df")

## DF PERSO*

In [ ]:
with open ("Entrees_BSPE-1921.tsv") as BSPE:
    df_perso = pd.read_csv(BSPE,sep='\t')

In [ ]:
df_perso

In [ ]:
#Retirer la colonne"Unnamed":
df_perso.drop('Unnamed: 0',
  axis='columns', inplace=True)

In [ ]:
#Nouvelle colonne pour la section : 
df_perso["article_section"] = df_perso.apply(lambda _: '', axis=1)

# print("ancien ordre colonnes : \n\n", list(df_perso.columns))

df_perso = df_perso[['article_id',
 'article_auteur', 
 'article_section',
 'article_titre',
 'article_page',
 'revue_annee',
 'revue_numero',]]

#Modification du article_id pour mettre les années plutôt que les numéros (meilleure lisibilité trans-revues)
for i in range (len(df_perso)):
    df_perso.iloc[i,0] = "BSPE-"+str(df_perso.iloc[i,5])+"-"+str(df_perso.iloc[i,4])

In [ ]:
df_perso

## Fusionner les deux DF : df_perso et df_Medica_court 

Points pour les merge : 
* article_id => On ne conserve que ceux correpsondant à df_perso ! => how='inner' on='article_id'
* revue_annee



In [ ]:
# df_BSPE_Merged = pd.merge(df_perso,df_Medica_court,
#         how='inner', on='article_id')

df_BSPE_Merged=df_perso.merge(df_Medica_court, how='inner', on='article_id')
df_BSPE_Merged #Ici on a des doublons car 2531 entrées au lieu de 2443...

In [ ]:
df_BSPE_Merged=df_BSPE_Merged.drop_duplicates(subset="article_id",keep="first")
#Pb : ici j'ai une perte d'informations car seulelemtn 2375 entrées au lieu de 2443

In [ ]:
df_BSPE_Merged

In [ ]:
df_perso.iloc[44,0:3]

Ajout des colonnes Section et Séance 

In [ ]:
#

for i in range (len(df_perso)):
    #Remplir la colonne des "auteurs4"
    if re.search(r"\(Discussion\)",df_perso.iloc[i,3]):
        df_perso.iloc[i,2]= "Discussion"
        df_perso.iloc[i,3]= re.sub(r"\(Discussion\)(.*)",r'\1',df_perso.iloc[i,3])
